In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

Generate data from stable AR(2) process
---

In [ ]:
# Parameters

mu = 0.
beta_true = np.array([0.7, 0.25])

noise_obs = 0.1
T = 128
N= 50
p = 2

In [ ]:
# Generate synthetic data
x_true = np.random.randn(T+1,N)*noise_obs
for t in xrange(p, T):
    x_true[t] += beta_true.dot(x_true[t-p:t][::-1])

In [ ]:
plt.plot(x_true);

Edward model
---

In [ ]:
from edward.models import Normal, InverseGamma, PointMass

mu = Normal(mu=0., sigma=10.)
beta = [Normal(mu=0., sigma=2.) for i in xrange(p)]

noise_proc = tf.constant(0.1) #InverseGamma(alpha=1.0, beta=1.0)
noise_obs = tf.constant(0.1) #InverseGamma(alpha=1.0, beta=1.0)

x = [0] * T
for n in xrange(p):
    x[n] = Normal(mu=mu*tf.ones([N]), sigma=10.0*tf.ones([N]))  # fat prior on x
for n in xrange(p, T):
    mu_ = mu
    for j in xrange(p):
        mu_ += beta[j] * x[n-j-1]
    x[n] = Normal(mu=mu_*tf.ones([N]), sigma=noise_proc*tf.ones([N]))

# use variance as zero to convert the list of x's to something Inference
# will understand
#y = Normal(mu=x, sigma=tf.constant(0.))


### MAP Inference

In [ ]:
print("setting up distributions")
qmu = PointMass(params=tf.Variable(0.))
qbeta = [PointMass(params=tf.Variable(0.)) for i in xrange(p)]
print("constructing inference object")
vdict = {mu: qmu}
vdict.update({b: qb for b, qb in zip(beta, qbeta)})
inference = ed.MAP(vdict, data={xt: xt_true for xt, xt_true in zip(x, x_true)})
print("running inference")
inference.run()

print("parameter estimates:")
print("beta: ", [qb.value().eval() for qb in qbeta])
print("mu: ", qmu.value().eval())

### Variational inference

In [ ]:
print("setting up variational distributions")
qmu = Normal(mu=tf.Variable(0.), sigma=tf.nn.softplus(tf.Variable(0.)))
qbeta = [Normal(mu=tf.Variable(0.), sigma=tf.nn.softplus(tf.Variable(0.))) for i in xrange(p)]
print("constructing inference object")
vdict = {mu: qmu}
vdict.update({b: qb for b, qb in zip(beta, qbeta)})
inference_vb = ed.KLqp(vdict, data={xt: xt_true for xt, xt_true in zip(x, x_true)})
print("running inference")
inference_vb.run()

print("parameter estimates:")
for j in xrange(p):
    print("beta[%d]: " % j, qbeta[j].mean().eval(), " +- ", qbeta[j].std().eval())
print("mu: ", qmu.variance().eval(), qmu.std().eval())

### Compare with estimate from statsmodels

In [ ]:
from statsmodels.tsa import ar_model

mu_res = np.empty(N)
beta_res = np.empty((N, p))
for n in xrange(N):
    ar2_sm = ar_model.AR(x_true[:,n])
    res = ar2_sm.fit(maxlag=2, ic=None, trend='c')
    mu_res[n] = res.params[0]
    beta_res[n] = res.params[1:]


print("statsmodels AR(2) params:")
print("mu: ", np.mean(mu_res))
print("beta: ", np.mean(beta_res, axis=0))